[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [1]:
# Imports
from getpass import getpass
import pandas as pd
import os

In [2]:
!pip install geopandas
import geopandas as gpd

# Data Preparation

In [3]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull 
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

Already up to date.


## Load toy data

In [4]:
planned, d_2016_10_21 = pd.read_excel("PGM-Project/data/toy.xlsx", sheet_name=[0,1]).values()

## Download MBTA Data

In [5]:
if not os.path.exists("MBTA Bus Ridership.geojson"):
  !wget https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
  !mv eec03d901d2e470ebd5758c60d793e8e_0.geojson "MBTA Bus Ridership.geojson"


In [6]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1"):
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv"):
  !unzip data.1

## Load MBTA Data

### MBTA Bus Ridership

In [8]:
df = gpd.read_file("MBTA Bus Ridership.geojson")

In [9]:
df

,FID,season,route_id,route_variant,direction_id,trip_start_time,day_type_id,day_type_name,stop_name,stop_id,stop_sequence,boardings,alightings,load_,sample_size,geometry
0,1,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,WASHINGTON ST OPP RUGGLES ST,1,2,0.4,0.3,7.8,13,None
1,2,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,ALBANY ST OPP RANDALL ST,10003,5,0.1,0.0,9.4,13,None
2,3,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ SIDNEY ST,101,19,0.0,0.4,10.4,13,None
3,4,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ PROSPECT,102,20,0.6,2.5,8.5,13,None
4,5,Fall 2016,1,1-_-0,0,04:37:00,day_type_01,weekday,MASSACHUSETTS AVE @ BIGELOW S,104,21,0.1,0.2,8.4,13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451727,3451728,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,HIGHLAND AVE @ PINE ST,5062,9,0.0,0.0,0.0,5,None
3451728,3451729,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ DEXTER ST,5066,15,0.0,0.0,0.0,5,None
3451729,3451730,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,CLIFTON ST @ CEDAR ST,5067,16,0.0,0.0,0.0,5,None
3451730,3451731,Fall 2019,99,99-8-1,1,22:30:00,day_type_02,saturday,SUMMER ST @ CLIFTON ST,5068,17,0.0,0.0,0.0,5,None


### MBTA Bus Arrival

In [7]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")

In [10]:
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


## Selection of arrival bus route

In [11]:
from operator import add
import glob

del df2

bus_arrival_files = glob.glob('./*.csv')

df_cumulative = pd.DataFrame()
for f in bus_arrival_files:
  df_tmp = pd.read_csv(f)
  df_cumulative = df_cumulative.append(df_tmp[['route_id','direction','stop_id']])
  print(f"Loaded {f}")

del df_tmp

Loaded ./MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded ./MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2019.csv
Loaded ./MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded ./MBTA Bus Arrival Departure Jan-Mar 2019.csv


In [12]:
# Calculate visits per stop
group = df_cumulative[['route_id','direction','stop_id']].groupby(['route_id','direction'])
unique_stops = group.nunique().rename(columns={"stop_id":"unique_stop_ids"})
total_stops = group.sum().rename(columns={"stop_id":"total_stops"})
stops = pd.merge(unique_stops, total_stops, left_index=True, right_index=True)
stops['visits_per_stop'] = stops['total_stops'] / stops['unique_stop_ids']

In [13]:
stops.sort_values('visits_per_stop', ascending=False).head(15)

unique_stop_ids  total_stops  visits_per_stop
route_id direction                                               
SL2      Outbound                 6  21390875994     3.565146e+09
         Inbound                  6  20743805635     3.457301e+09
SL1      Outbound                 7  21743224366     3.106175e+09
743      Outbound                11  31095228505     2.826839e+09
746_     Inbound                  4   9681659547     2.420415e+09
743      Inbound                 13  30649632094     2.357664e+09
746_     Outbound                 4   8298232654     2.074558e+09
SL1      Inbound                 13  26595214107     2.045786e+09
39       Outbound                10  11792808089     1.179281e+09
SL5      Outbound                12  13589567176     1.132464e+09
39       Inbound                 11  10001519728     9.092291e+08
42       Outbound                 4   2729267865     6.823170e+08
23       Outbound                12   7419938681     6.183282e+08
28       Inbound                 11   6427163857     5.842876e+08
23       Inbound                 10   5810033012     5.810033e+08

## Selection of ridership bus route

In [14]:
tot_sample_size = df.groupby(by=['route_id', 'direction_id'], as_index=False)['sample_size'].sum()
tot_sample_size.sort_values(by = 'sample_size')

,route_id,direction_id,sample_size
273,725,0,1040
274,725,1,1100
211,502,0,3187
57,170,1,4200
213,503,0,4581
...,...,...,...
120,32,1,746150
248,66,1,754626
1,1,1,758696
112,28,1,827974


# Preprocess data

## Toy 

In [15]:
planned.columns

Index(['Ligne', 'De', 'Début', 'Fin', 'À', 'NoPermanent', 'Voiture', 'Type',
       'Direction'],
      dtype='object')

In [16]:
filter_rows = (planned['Ligne'] == 10.0) & \
              (planned['Direction'] == "Nord")

filter_columns = ['Ligne', 'De', 'Début', 'Fin', 'À', 'NoPermanent', 'Direction']

planned[filter_rows][filter_columns]

,Ligne,De,Début,Fin,À,NoPermanent,Direction
2,10.0,M15001,2000-01-01 20:21:00,2000-01-01 17:39:00,H052,25286880,Nord
4,10.0,M15001,2000-01-01 22:21:00,2000-01-01 16:18:00,H052,80018209,Nord
6,10.0,M15001,2000-01-02 00:21:00,2000-01-01 14:55:00,H052,79994754,Nord
17,10.0,M15001,2000-01-02 11:21:00,2000-01-01 08:12:00,H052,25286806,Nord
19,10.0,M15001,2000-01-02 13:21:00,2000-01-01 07:02:00,H052,33657803,Nord
21,10.0,M15001,2000-01-02 15:21:00,2000-01-01 05:59:00,H052,25286802,Nord


In [17]:
filter_rows = (d_2016_10_21['ROUTE_ID'] == 10) & \
              (d_2016_10_21['DIRECTION'] == 1)

filter_columns = ['MEA_DATE', 'TRIP_ID', 'ROUTE_ID',
       'DIRECTION', 'STOP_ID', 'SCHEDULED_TIME',
       'MEA_ARR_TIME', 'MEA_DEP_TIME', 'NB_BOARDING', 'NB_DEBARKING',
       'PASSENGER_LOAD', 'AVG_PASSENGER_LOAD_TRIP', 'SYSRECNO']

d_2016_10_21[filter_rows][filter_columns]

,MEA_DATE,TRIP_ID,ROUTE_ID,DIRECTION,STOP_ID,SCHEDULED_TIME,MEA_ARR_TIME,MEA_DEP_TIME,NB_BOARDING,NB_DEBARKING,PASSENGER_LOAD,AVG_PASSENGER_LOAD_TRIP,SYSRECNO
0,42664,23,10,1,120263,1092600,1092790,1093000,0,16,0,36.0,101471030
1,42664,23,10,1,108481,1073400,1066570,1073500,1,0,1,NaN,101471031
4,42664,21,10,1,120263,1042200,1040910,1050880,0,12,0,39.0,101471034
5,42664,21,10,1,108481,1021200,1017790,1021520,0,0,0,NaN,101471035
8,42664,19,10,1,120263,991800,988900,1001040,0,20,0,41.0,101471038
9,42664,19,10,1,108481,970800,971500,972090,0,0,0,NaN,101471039
12,42664,8,10,1,120263,753600,752210,752370,0,12,0,36.0,101471042
13,42664,8,10,1,108481,732600,726450,732740,4,0,4,NaN,101471043
16,42664,6,10,1,120263,706800,708860,710930,0,26,0,51.0,101471046
17,42664,6,10,1,108481,686400,686530,687930,0,0,0,NaN,101471047
